# Introduction
Kickstarter.....

# Objective

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sb

In [2]:
# Read in data
project = pd.read_csv('../Data/ks-projects-201801.csv')
project.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [ ]:
project.info()
# The dataset has 378661 rows
# At a glance, only the attributes 'name' and 'usd pledged' are missing data
# 43.3 MB!!!!

In [128]:
pd.options.display.float_format = '{:20,.2f}'.format
project.describe()
# The max row is fascinating, find the entries that correspond later.

,goal,pledged,backers,usd_pledged_real,usd_goal_real
count,"378,661.00","378,661.00","378,661.00","378,661.00","378,661.00"
mean,"49,080.79","9,682.98",105.62,"9,058.92","45,454.40"
std,"1,183,391.26","95,636.01",907.19,"90,973.34","1,152,950.06"
min,0.01,0.00,0.00,0.00,0.01
25%,"2,000.00",30.00,2.00,31.00,"2,000.00"
50%,"5,200.00",620.00,12.00,624.33,"5,500.00"
75%,"16,000.00","4,076.00",56.00,"4,050.00","15,500.00"
max,"100,000,000.00","20,338,986.27","219,382.00","20,338,986.27","166,361,390.71"


In [132]:
project[project.goal == project.goal.max()].head()
project[project.goal == project.goal.min()]

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged_real,usd_goal_real
304489,LOVELAND Round 6: A Force More Powerful,Conceptual Art,Art,USD,2009-12-04,0.01,2009-11-25 07:54:49,100.00,successful,6,US,100.00,0.01
317771,"Word-of-mouth publishing: get ""Corruptions"" ou...",Fiction,Publishing,USD,2011-12-13,0.01,2011-11-07 16:46:52,0.00,canceled,0,US,0.00,0.01


### Attribute Descriptions:

- __ID__: Project ID
- __name__: Kickstarter Project name
- __category__: Sub-category to main_category
- __main_category__: Parent category, is split into sub-categories category
- __currency__: Currency of the project
- __deadline__: Deadline for the project to be fully funded
- __goal__: Goal amount in project curency
- __launched__: Date project was launched on kickstarter
- __pledged__: Amount pledged in the project currency
- __state__: The state of the project backing (failed, successful, cancelled, undefined, suspended)
- __backers__: Number of backers that have pledged/supported the project
- __country__: Country origin of project
- __usd_pledged__: Pledged amount in USD (conversion by KS)
- __usd_pedged_real__: Pledged amount in USD (conversion by fixe.io api)
- __usd_goal_real__: Goal amount in USD

# Part 1. Data Cleanup

#### Things to do still
1. Fill the missing na names values with 'category x' <font color=blue>DONE</font>
2. There are over 3000 usd_pledged na values <font color=blue>DONE, removed usd_pledged column as values look inaccurate</font>
    - See if there is any difference between usd_pledged and usd_pledged_real 
    - If no difference, i can probably remove the usd_pledged column
3. There are a number of country values that have the value N,0" <font color=blue>DONE updated values with currency</font>
    - See if there are any instances where currency != country
    - If each currency == country, then we can simply make the missing country values the currency country
    
    
4. There are a number of projects that have a state of undefined <font color=blue>Done, updated undefined to either be successful or failed</font>
    - Confirm the deadline has passed and if the pledged >= goal then set as successful
        if pledged < goal put failed
        if the deadline is past January 2018 (i believe), confrm what to put..... see if viable case.

In [3]:
# Remove the ID column as it provides no value in the analysis
# Rename 'usd pledged' to usd_pledged for consistency in headings

project.drop('ID', axis = 1, inplace = True)
project.rename(columns={'usd pledged': 'usd_pledged'}, inplace=True)
project.columns

Index(['name', 'category', 'main_category', 'currency', 'deadline', 'goal',
       'launched', 'pledged', 'state', 'backers', 'country', 'usd_pledged',
       'usd_pledged_real', 'usd_goal_real'],
      dtype='object')

In [4]:
# Find the null entries for name and usd pledged
na_name = project[project['name'].isna()]
na_name

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,usd_pledged_real,usd_goal_real
166851,NaN,Narrative Film,Film & Video,USD,2012-02-29,200000.0,2012-01-01 12:35:31,100.0,failed,1,US,100.00,100.00,200000.00
307234,NaN,Video Games,Games,GBP,2013-01-06,2000.0,2012-12-19 23:57:48,196.0,failed,12,GB,317.73,316.05,3224.97
309991,NaN,Product Design,Design,USD,2016-07-18,2500.0,2016-06-18 05:01:47,0.0,suspended,0,US,0.00,0.00,2500.00
338931,NaN,Painting,Art,USD,2011-12-05,35000.0,2011-11-06 23:55:55,220.0,failed,5,US,220.00,220.00,35000.00


In [5]:
# As there are only 4 projects missing names and they are all in different main categories,
# they will be renamed to 'Category X'
project['name'] = project['name'].fillna(project['category'] + ' X')

In [6]:
project.loc[[166851, 307234]]

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,usd_pledged_real,usd_goal_real
166851,Narrative Film X,Narrative Film,Film & Video,USD,2012-02-29,200000.0,2012-01-01 12:35:31,100.0,failed,1,US,100.00,100.00,200000.00
307234,Video Games X,Video Games,Games,GBP,2013-01-06,2000.0,2012-12-19 23:57:48,196.0,failed,12,GB,317.73,316.05,3224.97


In [7]:
project[project['name'].isna()]

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,usd_pledged_real,usd_goal_real


## 2: Fill NAs for usd_pledged

In [8]:
na_usd_pledged = project[project['usd_pledged'].isna()]
na_usd_pledged.head()

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,usd_pledged_real,usd_goal_real
169,STREETFIGHTERZ WHEELIE MURICA,Film & Video,Film & Video,USD,2014-09-20,6500.0,2014-08-06 21:28:36,555.00,undefined,0,"N,0""",NaN,555.00,6500.00
328,Duncan Woods - Chameleon EP,Music,Music,AUD,2015-08-25,4500.0,2015-08-04 12:05:17,4767.00,undefined,0,"N,0""",NaN,3402.08,3211.53
632,The Making of Ashley Kelley's Debut Album,Music,Music,USD,2015-04-09,3500.0,2015-03-10 20:06:13,3576.00,undefined,0,"N,0""",NaN,3576.00,3500.00
647,Butter Side Down Debut Album,Music,Music,USD,2015-11-26,6000.0,2015-11-02 22:09:19,7007.80,undefined,0,"N,0""",NaN,7007.80,6000.00
749,Chase Goehring debut EP,Music,Music,USD,2016-03-21,3000.0,2016-02-23 03:09:49,3660.38,undefined,0,"N,0""",NaN,3660.38,3000.00


In [9]:
na_count = project['usd_pledged'].isna().sum()
na_count

3797

In [10]:
project.loc[(project['country'] != 'US') & (project['backers'] > 0)].head()
# The USD pledged column seems to provide incorrect converted values
# Therefore the column will be removed for accuracy purposes

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,usd_pledged_real,usd_goal_real
14,Tombstone: Old West tabletop game and miniatur...,Tabletop Games,Games,GBP,2017-05-03,5000.0,2017-04-05 19:44:18,94175.00,successful,761,GB,57763.78,121857.33,6469.73
21,MikeyJ clothing brand fundraiser,Childrenswear,Fashion,AUD,2017-09-07,2500.0,2017-08-08 01:20:20,1.00,failed,1,AU,0.00,0.81,2026.10
23,Mountain brew: A quest for alcohol sustainability,Drinks,Food,NOK,2015-02-25,500.0,2015-01-26 19:17:33,48.00,failed,3,NO,6.18,6.29,65.55
27,Permaculture Skills,Webseries,Film & Video,CAD,2014-12-14,17757.0,2014-11-14 18:02:00,48905.00,successful,571,CA,43203.25,42174.03,15313.04
28,Rebel Army Origins: The Heroic Story Of Major ...,Comics,Comics,GBP,2016-01-28,100.0,2015-12-29 16:59:29,112.38,successful,27,GB,167.70,160.60,142.91


In [11]:
project.drop('usd_pledged', axis = 1, inplace = True)
project.columns

Index(['name', 'category', 'main_category', 'currency', 'deadline', 'goal',
       'launched', 'pledged', 'state', 'backers', 'country',
       'usd_pledged_real', 'usd_goal_real'],
      dtype='object')

## 2: Fill NAs for Country

In [18]:
project.loc[project.country == 'N,0"'].head()

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged_real,usd_goal_real


In [15]:
# Replace the value with the first two letters of the currency 
project.loc[project.country == 'N,0"', 'country'] = project['currency'].str[:2]

In [16]:
project.iloc[[168,328, 632, 647, 749]]

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged_real,usd_goal_real
168,Help me prepare for an upcoming art show,Painting,Art,USD,2016-04-24,1200.0,2016-03-10 21:25:41,2.00,failed,2,US,2.00,1200.00
328,Duncan Woods - Chameleon EP,Music,Music,AUD,2015-08-25,4500.0,2015-08-04 12:05:17,4767.00,undefined,0,AU,3402.08,3211.53
632,The Making of Ashley Kelley's Debut Album,Music,Music,USD,2015-04-09,3500.0,2015-03-10 20:06:13,3576.00,undefined,0,US,3576.00,3500.00
647,Butter Side Down Debut Album,Music,Music,USD,2015-11-26,6000.0,2015-11-02 22:09:19,7007.80,undefined,0,US,7007.80,6000.00
749,Chase Goehring debut EP,Music,Music,USD,2016-03-21,3000.0,2016-02-23 03:09:49,3660.38,undefined,0,US,3660.38,3000.00


In [17]:
project.loc[project.country == 'N,0"']

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged_real,usd_goal_real


## 3. Assign undefined state values
From above
state: The state of the project backing (failed, successful, cancelled, undefined, suspended)

In [52]:
project.loc[project.state == 'undefined'].head()
# 3562 entries

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged_real,usd_goal_real
169,STREETFIGHTERZ WHEELIE MURICA,Film & Video,Film & Video,USD,2014-09-20,6500.0,2014-08-06 21:28:36,555.00,undefined,0,US,555.00,6500.00
328,Duncan Woods - Chameleon EP,Music,Music,AUD,2015-08-25,4500.0,2015-08-04 12:05:17,4767.00,undefined,0,AU,3402.08,3211.53
632,The Making of Ashley Kelley's Debut Album,Music,Music,USD,2015-04-09,3500.0,2015-03-10 20:06:13,3576.00,undefined,0,US,3576.00,3500.00
647,Butter Side Down Debut Album,Music,Music,USD,2015-11-26,6000.0,2015-11-02 22:09:19,7007.80,undefined,0,US,7007.80,6000.00
749,Chase Goehring debut EP,Music,Music,USD,2016-03-21,3000.0,2016-02-23 03:09:49,3660.38,undefined,0,US,3660.38,3000.00


In [63]:
project.loc[(project.state != 'live') &
            (project.state != 'canceled') &
            (project.state != 'suspended')].deadline.max()

'2018-01-02'

In [85]:
#project.loc[(project.state == 'undefined')].deadline.max()
project.loc[project.deadline == project.loc[project.state=='undefined'].deadline.max()].head()
# The most recent deadline with a state of undefined is 06/09/2016
# which is a year and a half behind the most up to date data
# Therefore we do not need to check whether any undefined projects have not yet passed the deadline yet.
# We can assume all projects with a state of undefined are no longer live

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged_real,usd_goal_real
76,"""It's Complicated"" by Ariana Salome",Ready-to-wear,Fashion,USD,2016-06-09,68000.0,2016-05-10 22:52:00,0.0,failed,0,US,0.0,68000.0
2384,Off Constantly's Emmy Episode,Television,Film & Video,USD,2016-06-09,1000.0,2016-05-10 02:14:52,80.0,failed,2,US,80.0,1000.0
4795,Elvenroot Collapsible Dice Tower & Dice Case,Tabletop Games,Games,USD,2016-06-09,15000.0,2016-05-10 21:14:08,29206.0,successful,194,US,29206.0,15000.0
4988,Treefinger Productions,Immersive,Theater,USD,2016-06-09,50000.0,2016-04-20 00:00:18,0.0,failed,0,US,0.0,50000.0
5405,Raptured Apparel,Apparel,Fashion,USD,2016-06-09,3000.0,2016-05-10 00:16:25,1.0,failed,1,US,1.0,3000.0


In [72]:
project['state'].value_counts()
# As the vast majority of projects were either failed or successful, 
# the projects with a state of undefined will be either set to those two values
# This will not affect the distribution of data too much

failed        197719
successful    133956
canceled       38779
undefined       3562
live            2799
suspended       1846
Name: state, dtype: int64

In [109]:
test_undefined = project.loc[project.state == 'undefined'].copy()
test_undefined

# row 328, 632, 647 give true
# row 169 is false

test_undefined['state'].loc[test_undefined.pledged >= test_undefined.goal] = 'successful'
test_undefined['state'].loc[test_undefined.pledged < test_undefined.goal] = 'failed'

#test_undefined.state = 'successful'.where(test_undefined.pledged >= test_undefined.goal, inplace = True)
#test_undefined.state = 'failed'.where(test_undefined.pledged < test_undefined.goal, inplace=True)

In [112]:
test_undefined.iloc[[1,2,3,4]]
#test_undefined.iloc[[0,11]]

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged_real,usd_goal_real
328,Duncan Woods - Chameleon EP,Music,Music,AUD,2015-08-25,4500.0,2015-08-04 12:05:17,4767.00,successful,0,AU,3402.08,3211.53
632,The Making of Ashley Kelley's Debut Album,Music,Music,USD,2015-04-09,3500.0,2015-03-10 20:06:13,3576.00,successful,0,US,3576.00,3500.00
647,Butter Side Down Debut Album,Music,Music,USD,2015-11-26,6000.0,2015-11-02 22:09:19,7007.80,successful,0,US,7007.80,6000.00
749,Chase Goehring debut EP,Music,Music,USD,2016-03-21,3000.0,2016-02-23 03:09:49,3660.38,successful,0,US,3660.38,3000.00


In [120]:
project['state'].loc[(project.state == 'undefined') & (project.pledged >= project.goal)] = 'successful' # 1654
project['state'].loc[(project.state == 'undefined') & (project.pledged < project.goal)] = 'failed'# 1908

In [122]:
project.iloc[[328, 632, 647]] # successful
project.iloc[[169, 1026, 1036]] # failed

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged_real,usd_goal_real
328,Duncan Woods - Chameleon EP,Music,Music,AUD,2015-08-25,4500.0,2015-08-04 12:05:17,4767.0,successful,0,AU,3402.08,3211.53
632,The Making of Ashley Kelley's Debut Album,Music,Music,USD,2015-04-09,3500.0,2015-03-10 20:06:13,3576.0,successful,0,US,3576.00,3500.00
647,Butter Side Down Debut Album,Music,Music,USD,2015-11-26,6000.0,2015-11-02 22:09:19,7007.8,successful,0,US,7007.80,6000.00


In [123]:
project.loc[project.state == 'undefined']

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged_real,usd_goal_real


In [125]:
# Verify there are no longer any missing values
project.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 13 columns):
name                378661 non-null object
category            378661 non-null object
main_category       378661 non-null object
currency            378661 non-null object
deadline            378661 non-null object
goal                378661 non-null float64
launched            378661 non-null object
pledged             378661 non-null float64
state               378661 non-null object
backers             378661 non-null int64
country             378661 non-null object
usd_pledged_real    378661 non-null float64
usd_goal_real       378661 non-null float64
dtypes: float64(4), int64(1), object(8)
memory usage: 37.6+ MB


In [ ]:
# THE DATASET HAS BEEN CLEANED UP
# NOW WE CAN MOVE ON TO THE ANALYSIS

# Part 2. Data Analysis